### 1. Timing

* How long does it take for a function to run?

```
* The inner function (“wrapper”) will run the original function 
* But it’ll keep track of the time before and after doing so.
* Before returning the result to the user, write the timing information to a logfile
```

In [1]:
import time

In [2]:
def logtime(func):
    
    def wrapper(*args, **kwargs):
        
        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time
        
        with open('timelog.txt', 'a') as outfile: 
            outfile.write(f'{time.time()}\t{func.__name__}\t{total_time}\n')
            
        return result 
    
    return wrapper

In [3]:
@logtime

def slow_add(a, b): 
    time.sleep(2)
    return a + b


In [4]:
@logtime

def slow_mul(a, b):     
    time.sleep(3)    
    return a * b

In [5]:
# test function
slow_add(100,100)

200

records from timelogs:

1565464610.880044	slow_add	2.0048961639404297



### 2. Run function once per minute

* Raise an exception if we try to run a function more than once in 60 seconds

In [6]:
def once_per_minute(func): 
    
    last_invoked = 0
    
    def wrapper(*args, **kwargs):
        
        nonlocal last_invoked
        
        elapsed_time = time.time() - last_invoked 
        
        if elapsed_time < 60:
            
            raise RuntimeError(f"Only {elapsed_time} has passed") 
            
        last_invoked = time.time()
        
        return func(*args, **kwargs)
    
    return wrapper

In [7]:
@once_per_minute

def add(a, b): 
    return a + b

In [8]:
add(2,3)

5

In [9]:
add(2,3)

RuntimeError: Only 0.011683225631713867 has passed

### 3. Perform function per N seconds
* Raise an exception if we try to run a function more than once in n seconds

In [10]:
def once_per_n(n): 
    
    def middle(func):
        last_invoked = 0
        
        def wrapper(*args, **kwargs): 
            nonlocal last_invoked
            elapsed_time = time.time() - last_invoked
            if elapsed_time < n:
                raise RuntimeError(f"Only {elapsed_time} has passed")
                
            last_invoked = time.time()
            return func(*args, **kwargs) 
        
        return wrapper
    
    return middle

In [11]:
@once_per_n(10) 

def add(a, b):
    return a + b

In [12]:
add(2,3)

5

In [13]:
add(2,3)

RuntimeError: Only 0.013686656951904297 has passed

### 4. Memoization
* Cache the results of function calls, so we don’t need to call them again

In [15]:
import pickle

In [16]:
# Executes each time the decorated function is executed
def memoize(func): 
    cache = {}
    
    # Executes each time the decorated function is executed
    def wrapper(*args, **kwargs): 
        t = (pickle.dumps(args), pickle.dumps(kwargs)) 
        if t not in cache:
            print(f"Caching NEW value for {func.__name__}{args}") 
            cache[t] = func(*args, **kwargs)
        else:
            print(f"Using OLD value for {func.__name__}{args}") 
            
        return cache[t]
    
    return wrapper

In [17]:
@memoize
def add(a, b): 
    print("Running add!") 
    return a + b


In [18]:
@memoize
def mul(a, b): 
    print("Running mul!") 
    return a * b

In [19]:
add(2,2)

Caching NEW value for add(2, 2)
Running add!


4

In [20]:
add(2,2)

Using OLD value for add(2, 2)


4

### 5. Attributes
* Give many objects the same attributes, but without using inheritance

In [21]:
def fancy_repr(self):
    return f"I'm a {type(self)}, with vars {vars(self)}"

In [22]:
def better_repr(c):
    c.__repr__ = fancy_repr
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        return o 
    return wrapper

In [23]:
# Alternative approach
def better_repr(c): #The decorated class
    c.__repr__ = fancy_repr 
    return c #Return a callable — class

In [24]:
@better_repr
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [25]:
f = Foo(10, [10, 20, 30])
print(f)

I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30]}


* The **@object_birthday** decorator, when applied to a class, will add a new **created_at** attribute to new objects 
* Give every object its own birthday. This will contain the timestamp at which each instance was created

In [26]:
def object_birthday(c): #The decorated class
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs) 
        o._created_at = time.time() 
        return o
    return wrapper


In [27]:
@object_birthday
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [28]:
f = Foo(10, [10, 20, 30]) 
print(f) 
print(f._created_at)

1565719157.690739


In [29]:
def object_birthday(c): 
    c.__repr__ = fancy_repr  #Add a method to the class
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs) 
        o._created_at = time.time()  #Add an attribute to the instance
        return o
    return wrapper

In [30]:
@object_birthday
class Foo():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [31]:
f = Foo(10, [10, 20, 30]) 
print(f) 


I'm a <class '__main__.Foo'>, with vars {'x': 10, 'y': [10, 20, 30], '_created_at': 1565719157.728944}
